In [36]:
import torch
from torch import nn 

In [37]:
# Создайте случайный FloatTensor размера 3x4x5
tensor = torch.FloatTensor(3, 4, 5)
tensor

tensor([[[ 1.4013e-45,  0.0000e+00, -1.4123e-31,  5.6753e-43, -2.0146e-36],
         [ 5.6753e-43,  0.0000e+00,  0.0000e+00, -1.4469e-31,  0.0000e+00],
         [-1.3346e-33,  5.6753e-43, -4.2560e-30,  5.6753e-43,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  1.4013e-45,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  7.4269e-44,  0.0000e+00,  1.4013e-45,  0.0000e+00]],

        [[ 9.1844e-41,  1.1551e-40,  2.2960e-41,  4.1478e-43,  0.0000e+00],
         [ 0.0000e+00, -1.5273e+14, -1.3632e-41, -4.5536e-36,  4.5908e-41],
         [ 0.0000e+00,  0.0000e+00,  1.4013e-45,  0.0000e+00, -1.4122e-31],
         [ 5.6753e-43, -1.4374e-30,  5.6753e-43,  0.0000e+00,  0.0000e+00]]])

In [38]:
# Выведите его форму (shape)

In [39]:
tensor.shape

torch.Size([3, 4, 5])

In [40]:
# Приведите его к форме 6 X 10
tensor.reshape(6, 10)

tensor([[ 1.4013e-45,  0.0000e+00, -1.4123e-31,  5.6753e-43, -2.0146e-36,
          5.6753e-43,  0.0000e+00,  0.0000e+00, -1.4469e-31,  0.0000e+00],
        [-1.3346e-33,  5.6753e-43, -4.2560e-30,  5.6753e-43,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  1.4013e-45,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  7.4269e-44,  0.0000e+00,  1.4013e-45,  0.0000e+00],
        [ 9.1844e-41,  1.1551e-40,  2.2960e-41,  4.1478e-43,  0.0000e+00,
          0.0000e+00, -1.5273e+14, -1.3632e-41, -4.5536e-36,  4.5908e-41],
        [ 0.0000e+00,  0.0000e+00,  1.4013e-45,  0.0000e+00, -1.4122e-31,
          5.6753e-43, -1.4374e-30,  5.6753e-43,  0.0000e+00,  0.0000e+00]])

In [41]:
# Умножьте его на вектор [1, 4, 2, 2, 1] поэлементно
vector = torch.FloatTensor([1, 4, 2, 2, 1])
tensor.mul(vector)

tensor([[[ 1.4013e-45,  0.0000e+00, -2.8247e-31,  1.1351e-42, -2.0146e-36],
         [ 5.6753e-43,  0.0000e+00,  0.0000e+00, -2.8938e-31,  0.0000e+00],
         [-1.3346e-33,  2.2701e-42, -8.5121e-30,  1.1351e-42,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  5.6052e-45,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  2.9708e-43,  0.0000e+00,  2.8026e-45,  0.0000e+00]],

        [[ 9.1844e-41,  4.6204e-40,  4.5921e-41,  8.2957e-43,  0.0000e+00],
         [ 0.0000e+00, -6.1093e+14, -2.7264e-41, -9.1072e-36,  4.5908e-41],
         [ 0.0000e+00,  0.0000e+00,  2.8026e-45,  0.0000e+00, -1.4122e-31],
         [ 5.6753e-43, -5.7496e-30,  1.1351e-42,  0.0000e+00,  0.0000e+00]]])

In [42]:
# Умножьте тензор матрично на себя, чтобы результат был размерности 6x6
a = torch.matmul(tensor.reshape(6,10), tensor.reshape(10,6))
display(a)
a.shape

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [-1.1343e-29,  0.0000e+00, -2.1402e-31,  0.0000e+00, -1.1946e-26,
         -1.7642e-26],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -8.6679e-29,
          0.0000e+00]])

torch.Size([6, 6])

In [43]:
# Посчитайте производную функции y = x**3 + z - 75t в точке (1, 0.5, 2)
x = torch.tensor(1., requires_grad=True)
z = torch.tensor(0.5, requires_grad=True)
t = torch.tensor(2., requires_grad=True)

def define_func(x=x,z=z,t=t):
    return x**3 + z - 75*t

for i in [x,z,t]:
    y = define_func()
    y.backward(i)
    print(f"Производная в точке {i}: ", i.grad)

Производная в точке 1.0:  tensor(3.)
Производная в точке 0.5:  tensor(1.5000)
Производная в точке 2.0:  tensor(-262.5000)


In [44]:
# Создайте единичный тензор размера 5x6
tensor1 = torch.ones(5, 6)
tensor1

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])

In [45]:
# Переведите его в формат numpy
tensor1.numpy()

array([[1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.]], dtype=float32)

In [46]:
# Давайте теперь пооптимизируем: возьмите функцию y = x**w1 - 2 * x**2 + 5
# Посчитайте 

### Что посчитать-то? Почему нельзя задания по-нормальному описывать?!